In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import logging
from decorators import timer
from decorators import debug

logger = logging.getLogger(__name__)
logger.setLevel(level=logging.DEBUG)

pd.set_option('display.max_rows', 1000)
KEY = 'Z5YWd/GtQmvP+c3KedLUdJf0hRk8fzskVhf//D6mICa2X/yeW+OOp9MNnVHorH2WysKvgsBpltRibqv2Dx+V+w=='
url = 'http://apis.data.go.kr/1611000/nsdi/IndvdLandPriceService/attr/getIndvdLandPriceAttr'
params ={'serviceKey' : KEY, 'pnu' : '1111010200', 'stdrYear' : '2022', 'format' : 'xml', 'numOfRows' : '100', 'pageNo' : '4' }

response = requests.get(url, params=params)
soup = bs(response.text, features='xml')
numRows = int(soup.find('totalCount').text)
# print(response.text)

In [49]:
class InstallCPS:
    def __init__(self, KEY):
        self.params = {'serviceKey' : KEY, 'pnu' : None, 'stdrYear' : None, 'format' : 'xml', 'numOfRows' : 1000, 'pageNo' : '1' }
        self.url = 'http://apis.data.go.kr/1611000/nsdi/IndvdLandPriceService/attr/getIndvdLandPriceAttr'
        
    @timer
    def get_page(self, pnu:str, pageNo:int, year:int=2021):
        self.params['pageNo'] = str(pageNo)
        self.params['pnu'] = str(pnu)
        self.params['stdrYear'] = year
                
        response = requests.get(self.url, params = self.params)

        # Parsing the XML response to get the number of rows --> if there exists no data, numRows = 0
        soup = bs(response.text, features='xml')
        numRows = int(soup.find('totalCount').text)

        # Return the result dataframe if there is any record in the given year
        if numRows > 0:
            logger.info(msg=f"There is {numRows} records in {year}")
            return pd.read_xml(response.text, xpath=".//fields/field")

        # Return the result of the right previous year if there is no record in the given year
        return self.get_page(pnu, pageNo, year-1)


In [50]:
a = InstallCPS(KEY)

In [51]:
result = a.get_page('1111010200', 1)

get_page took 0.5424911975860596 seconds


In [53]:
result.head()

,pnu,ldCode,ldCodeNm,regstrSeCode,regstrSeCodeNm,mnnmSlno,stdrYear,stdrMt,pblntfPclnd,pblntfDe,stdLandAt,lastUpdtDt
0,1111010200100010001,1111010200,서울특별시 종로구 신교동,1,일반,1-1,2021,1,4092000,2021-05-31,N,2021-11-11
1,1111010200100010004,1111010200,서울특별시 종로구 신교동,1,일반,1-4,2021,1,4079000,2021-05-31,N,2021-11-11
2,1111010200100010007,1111010200,서울특별시 종로구 신교동,1,일반,1-7,2021,1,429400,2021-05-31,N,2021-11-11
3,1111010200100010008,1111010200,서울특별시 종로구 신교동,1,일반,1-8,2021,1,385600,2021-05-31,N,2021-11-11
4,1111010200100010009,1111010200,서울특별시 종로구 신교동,1,일반,1-9,2021,1,385600,2021-05-31,N,2021-11-11
